In [1]:
import pandas as pd
import numpy as np
import json
import gc

In [2]:
X = pd.read_csv('../treated_data/df_merged.csv', index_col=0)

with open('../treated_data/type_of_feature2.json', 'r') as fp:
    type_of_feature = json.load(fp)

/home/mateus/anaconda3/envs/home-credit/lib/python3.8/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


# Static DF train

In [18]:
# Create static df
df_static = X[type_of_feature['static']]
df_static.dtypes.unique()

array([dtype('int64'), dtype('O'), dtype('float64')], dtype=object)

# feature engineering
Positive days means feature, makes no sense


In [19]:
df_static['DAYS_EMPLOYED'].replace(365243,np.nan, inplace=True)
df_static['CREDIT_INCOME_PERCENT'] = df_static['AMT_CREDIT'] / df_static['AMT_INCOME_TOTAL']
df_static['ANNUITY_INCOME_PERCENT'] = df_static['AMT_ANNUITY_x'] / df_static['AMT_INCOME_TOTAL']
df_static['CREDIT_TERM'] = df_static['AMT_ANNUITY_x'] / df_static['AMT_CREDIT']
df_static['DAYS_EMPLOYED_PERCENT'] = df_static['DAYS_EMPLOYED'] / df_static['DAYS_BIRTH']

<ipython-input-19-390bf92115c8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_static['CREDIT_INCOME_PERCENT'] = df_static['AMT_CREDIT'] / df_static['AMT_INCOME_TOTAL']
<ipython-input-19-390bf92115c8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_static['ANNUITY_INCOME_PERCENT'] = df_static['AMT_ANNUITY_x'] / df_static['AMT_INCOME_TOTAL']
<ipython-input-19-390bf92115c8>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

The definition that was given to static features is the same thought all the months. that's why drop_duplicates

get_dummies droping first to minimize redundant info

dropped columns with too much absent information

In [5]:
df_static.drop_duplicates('SK_ID_CURR',inplace=True)
df_static = pd.get_dummies(df_static, drop_first=True)
df_static = df_static[df_static.columns[df_static.isnull().mean() < 0.3]].set_index('SK_ID_CURR')

<ipython-input-5-77f77f8d95a2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_static.drop_duplicates('SK_ID_CURR',inplace=True)


In [6]:
train_columns = df_static.columns

Index(['TARGET', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT',
       'AMT_ANNUITY_x', 'AMT_GOODS_PRICE', 'REGION_POPULATION_RELATIVE',
       'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION',
       ...
       'FONDKAPREMONT_MODE_reg oper spec account',
       'HOUSETYPE_MODE_specific housing', 'HOUSETYPE_MODE_terraced house',
       'WALLSMATERIAL_MODE_Mixed', 'WALLSMATERIAL_MODE_Monolithic',
       'WALLSMATERIAL_MODE_Others', 'WALLSMATERIAL_MODE_Panel',
       'WALLSMATERIAL_MODE_Stone, brick', 'WALLSMATERIAL_MODE_Wooden',
       'EMERGENCYSTATE_MODE_Yes'],
      dtype='object', length=187)

# Saving static df

In [7]:
df_static.to_csv('../treated_data/df_static.csv')